In [1]:
import pandas as pd
import numpy as np
import re
from random import choice
import datetime

In [2]:
df = pd.read_csv('attacks.csv', encoding = "ISO-8859-1")

In [3]:
# df.info()

In [4]:
# Droping last 2 cols because they contain almost no information 99% is null
df.drop(columns = ['Unnamed: 22','Unnamed: 23'], inplace = True)

In [5]:
# df.info()

In [6]:
# It can be seen that rows without case number are null in every column
# df.loc[df['Case Number'].isnull()].info()


In [7]:
# droping no-Case Number rows
df = df.loc[df['Case Number'].notnull()]

In [8]:
# there are lots or rows empty, only with Case Number, I will delete those.
# df.loc[df['Date'].isnull()].info()

In [9]:
df = df.loc[df['Date'].notnull()]

In [10]:
# Now we have mostly rows with information, but some cleaning will continue.
# df.info()

In [11]:
# Analyzing Type column...
df['Type'] = df['Type'].astype('str')
# df['Type'].value_counts()

In [12]:
# boatomg, boating and boat shoul be the same
df['Type'] = df['Type'].apply(lambda x: re.sub('Boat.+','Boat',x))

In [13]:
# Sea disaster is not provoked by someone, so it should be an unprovoked incident
df['Type'] = df['Type'].apply(lambda x: re.sub('Sea Disaster','Unprovoked',x))

In [14]:
# nan and Questionable can be deleted because there are only 6 and add no value
df = df.loc[df['Type'].apply(lambda x: False if x in ['nan','Questionable'] else True)]

In [15]:
# df['Type'].value_counts()

In [16]:
# In Boat incident and invalid incident it is unknown if it was provoked or not, so I will join them in unknown
df['Type'] = df['Type'].apply(lambda x: re.sub('Boat|Invalid','Unknown',x))

In [17]:
# Type column clean
# df['Type'].value_counts()

In [18]:
# Country column
df['Country'] = df['Country'].astype('str')

In [19]:
# All countries in caps
df['Country'] = df['Country'].apply(lambda x: x.upper())

In [20]:
df['Country'] = df['Country'].apply(lambda x: re.sub('.*SRI LANKA.*','SRI LANKA',x) if re.sub('.*SRI LANKA.*','SRI LANKA',x) else x)

In [21]:
# removing extra white spaces from both ends
df['Country'] = df['Country'].apply(lambda x: x.strip())

In [22]:
# Country column done
# df['Country'].value_counts()

In [23]:
# df.info()

In [24]:
# df['Area'].value_counts()

In [25]:
# cleaning Area column like Country col
df['Area'] = df['Area'].astype('str')
df['Area'] = df['Area'].apply(lambda x: x.title())
df['Area'] = df['Area'].apply(lambda x: x.strip())
# df['Area'].value_counts()

In [26]:
# Location has too many different values (4105), I don't think it has a lot of value, so i will remove it
# df['Location'].value_counts()

In [27]:
# removing Location col
df.drop(columns = ['Location'], axis = 1, inplace = True)

In [28]:
# Activity column
df['Activity'] = df['Activity'].astype('str')
df['Activity'] = df['Activity'].apply(lambda x: x.lower())

In [29]:
"""
There are too many with long descriptions, so I will choose top 150 acivities and if there 
is a word in the description apearing in those top 150 activities the description will become that word.
"""
acti_list = df['Activity'].unique().tolist()[:150]
acti_list = [i for i in acti_list if i != 'nan']

In [30]:
# acti_list

In [31]:
def act(activ):
    a = activ.split()
    for word in a:
        if word in acti_list:
            return word
    
    return 'other'

In [32]:
df['Activity'] = df['Activity'].apply(act)

In [33]:
# df['Activity'].value_counts()

In [34]:
# Name column has a lot of wrong names, it is very difficult to erase every name, so I will erase top 33
# not-names and place them as Unknown
df['Name'] = df['Name'].astype('str')
df['Name'] = df['Name'].apply(lambda x: x.strip().title())
names_to_erase = df['Name'].value_counts().index.tolist()[:34]

In [35]:
def erase(name):
    if name in names_to_erase:
        return 'Unknown'
    else:
        return name
    
df['Name'] = df['Name'].apply(erase)

In [36]:
# df.columns

In [37]:
# Sex column
df.rename(columns={'Sex ':'Sex'},inplace=True)
df['Sex'] = df['Sex'].astype('str')

In [38]:
# Non M or F characters will become M because they are aprox 10%, it will not make a big impact o data 
# df['Sex'].value_counts()

In [39]:
df['Sex'] = df['Sex'].apply(lambda x: 'M' if x != 'F' else 'F')

In [40]:
df['Age'] = df['Age'].astype('str').apply(lambda x: x.strip())

In [41]:
def age(string):
    if re.search('\d+', string):
        return re.search('\d+', string).group()
    else:
        return '0'
    
df['Age'] = df['Age'].apply(age)


In [42]:
df['Age'] = df['Age'].astype('int16')

In [43]:
# df.columns

In [44]:
# injury column is describing type of injury, so I will perform  basic cleaning and move on
df['Injury'] = df['Injury'].astype('str').apply(lambda x: x.strip().capitalize())

In [45]:
# Fatal (Y/N) column
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].astype('str').apply(lambda x: x.strip().upper())
# df['Fatal (Y/N)'].value_counts()

In [46]:
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].apply(lambda x: 'UNKNOWN' if (x == 'NAN' or x == '2017') else ('N' if x == 'M' else x))
# df['Fatal (Y/N)'].value_counts()

In [47]:
df.rename(columns={'Fatal (Y/N)':'Fatal'}, inplace = True)

In [48]:
# df.columns

In [49]:
# Time column will be resumed into 3 moments of day
df['Time']  = df['Time'].astype('str')

In [50]:
def time(string):
    hour = ''
    if re.search('\d\dh\d\d',string):
        hour =  re.search('\d\dh\d\d',string).group()[:2]
        if 12 <= int(hour) < 18:
            return 'Afternoon'
        if 18 <= int(hour) or int(hour) < 2:
            return 'Night'
        if 2 <= int(hour) < 12:
            return 'Morning'
        
    elif re.search('.*(afternoon|midday|evening).*',string.lower()):
        return 'Afternoon'
    elif re.search('.*morning.*',string.lower()):
        return 'Morning'
    elif re.search('.*(night|dusk).*',string.lower()):
        return 'Night'

    else:
        return choice(['Morning','Afternoon','Night'])

df['Time'] = df['Time'].apply(time)
# df['Time'].value_counts()

In [51]:
# Species column has too many different species input and 30% of data is missing, 
#I will drop it, because this col does not add a lot of value to df 

df.rename(columns = {'Species ':'Species'},inplace = True)
df['Species'].astype('str').value_counts()
df.drop(columns=['Species'], inplace = True)

In [52]:
# df.head()

In [53]:
# df.info()

In [54]:
# investigator or source column, we will erase null rows because there are less than 50
df = df.loc[df['Investigator or Source'].notnull()]

In [55]:
# df.info()

In [56]:
# href and href formula are almost exactly the same, they are equal in 6219 rows out of 6279, so we will only keep href
df.drop(columns = ['href formula'], axis=1, inplace = True)

In [57]:
# df.info()

In [58]:
# df.loc[df['Case Number.1'] == df['Case Number.2']] #6259 rows match
# df.loc[df['Case Number.1'] == df['Case Number']] #6256 rows match
# df.loc[df['Case Number'] == df['Case Number.2']] #6276 rows match

# We can get rid of las 2 cols because they almost add no value at all
df.drop(columns=['Case Number.1','Case Number.2'], axis = 1, inplace = True)

In [59]:
# Column original order is like a serial number for every incident, I think is valueble for the record.
df['original order'] = df['original order'].apply(lambda x: int(x)).astype('int16')

In [60]:
# df.info()

In [61]:
# year column, erase 2 null rows
df = df.loc[df['Year'].notnull()]

In [62]:
# df['Year'].value_counts()

In [63]:
df['Year'] = df['Year'].apply(lambda x: re.search('\d{1,4}',str(x)).group() if re.search('\d{1,4}',str(x)) else x)

In [64]:
df['Year'] = df['Year'].apply(lambda x: x if int(x) > 1500 else '1500')

In [65]:
df['Year'] = df['Year'].astype('int16')

In [66]:
# df

In [67]:
# date column
df['Date'] = df['Date'].astype('str').apply(lambda x: x.strip())

In [68]:
def date(d):
    if re.search('\d\d-\w{3}-\d\d\d\d',d):
        try:
            date = datetime.datetime.strptime(re.search('\d\d-\w{3}-\d\d\d\d',d).group(),'%d-%b-%Y')
        except:
            date = datetime.datetime.strptime(re.search('\d\d-\w{3}-\d\d\d\d',d).group(),'%d-%B-%Y')
        
    elif re.search('\d\d\d\d',d):
        string = re.search('\d\d\d\d',d).group()
        date = datetime.datetime.strptime(f'01-01-{string}','%d-%m-%Y')
    
    else:   
        return False
    
    return date.date()

# df.loc[df['Date'].apply(date).isnull()] # there are no null values

In [69]:
# Erasing 17 rows of strange dates
# df.loc[df['Date'].apply(date) == False].info()
df = df.loc[df['Date'].apply(date) != False]
df['Date'] = df['Date'].apply(date)

In [70]:
# Actually, year is not longer needed, because we can extracted fron date col
df.drop(columns = ['Year'], axis = 1,inplace = True)

In [71]:
# df.head()

In [73]:
# Exporting to csv
df.to_csv('file.csv',index = False)